In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.measure import compare_ssim
from os import listdir, path
from tempfile import TemporaryFile
import pymysql

%matplotlib inline

In [27]:
# Capturamos el vídeo
cap = cv2.VideoCapture('video1.mov')
frame_num = cap.get(cv2.CAP_PROP_FRAME_COUNT)
# Llamada al método
fgbg = cv2.createBackgroundSubtractorKNN(history=250, dist2Threshold=500, detectShadows=False)

# Deshabilitamos OpenCL, si no hacemos esto no funciona
cv2.ocl.setUseOpenCL(False)

lst_area = []
cont = 0
meh= []
while(1):
    # Leemos el siguiente frame
    ret, frame = cap.read()

    # Si hemos llegado al final del vídeo salimos
    if not ret:
        break
    
    # Reset variables
    a = 0
    temp_area = []
    
    # Aplicamos el algoritmo
    fgmask = fgbg.apply(frame)

    ALGO = cv2.dilate(fgmask, None, iterations=3) #PROBANDO

    # Copiamos el umbral para detectar los contornos
    contornosimg = ALGO.copy()

    # Buscamos contorno en la imagen
    im, contornos, hierarchy = cv2.findContours(contornosimg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    # Recorremos todos los contornos encontrados
    for c in contornos:
        # Eliminamos los contornos más pequeños
        if cv2.contourArea(c) < 5000:
            continue

        # Obtenemos el bounds del contorno, el rectángulo mayor que engloba al contorno
        (x, y, w, h) = cv2.boundingRect(c)
        # Dibujamos el rectángulo del bounds
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        a = a + 1
        temp_area.append(cv2.contourArea(c))
    
    if a > 0:
        cont = cont + 1
        if cont > 50:
            cont_act_mov = a
            cont = 0
            if len(lst_area) == 0:
                lst_area = temp_area
            else:
                for t in temp_area:
                    for j in lst_area:
                        meh.append(abs(t-j)/j)
                        if (abs(t-j)/j) > 1 :
                            lst_area.append(t)
    
    # Mostramos las capturas
    cv2.imshow('Camara',frame)
    cv2.imshow('Umbral',fgmask)
    cv2.imshow('Contornos',contornosimg)

    # Sentencias para salir, pulsa 's' y sale
    k = cv2.waitKey(1) & 0xff
    if k == ord("s"):
        break

# Liberamos la cámara y cerramos todas las ventanas
cap.release()
cv2.destroyAllWindows()


In [29]:
lst_area

[12240.5, 29710.5]

In [25]:
meh.sort()

In [26]:
meh

[0.0,
 0.003401360544217687,
 0.01549053356282272,
 0.025161871977706744,
 0.028891074502089994,
 0.029751659700024587,
 0.04241455618391935,
 0.058847635439718056,
 0.07233013687402672,
 0.08204245553643144,
 0.09745102860421277,
 0.10089336939595116,
 0.10650766330628637,
 0.11384312761249078,
 0.14666830587656748,
 0.1499467256782231,
 0.1533071059749201,
 0.1590033603802967,
 0.16084747151872797,
 0.16187197770674536,
 0.17195311859683632,
 0.17346938775510204,
 0.18133759527907548,
 0.18826325711007294,
 0.19502499795098763,
 0.19879518072289157,
 0.20203262027702648,
 0.20244242275223343,
 0.22383411195803624,
 0.2332185886402754,
 0.23756249487746905,
 0.2405950331940005,
 0.2422342430948283,
 0.2559626260142611,
 0.2567412507171543,
 0.25915908532087534,
 0.25977378903368575,
 0.25989672977624784,
 0.27645274977460865,
 0.299114826653553,
 0.30419637734611915,
 0.31411359724612736,
 0.31935906892877636,
 0.32370297516597,
 0.3239898368986149,
 0.32558806655192196,
 0.3268174739

In [35]:
lst_area = []
cont = 0

# Charge the video
camara = cv2.VideoCapture("video1.mov")

# # Charge the background
# background = cv2.VideoCapture("bg3.jpg")

# # Obtain the background
# (grabbed_b, frame_b) = background.read()
# fondo = cv2.cvtColor(frame_b, cv2.COLOR_BGR2GRAY)
# fondo = cv2.GaussianBlur(fondo, (21, 21), 0)

fondo = cv2.imread("bg3.jpg",cv2.IMREAD_GRAYSCALE)
fondo = cv2.GaussianBlur(fondo, (21, 21), 0)
meh = []
# Explore of the frames (photogram)
while True:
    # Obtain the frame
    (grabbed, frame) = camara.read()

    # If all the frames have been studied
    if not grabbed:
        break
        
    a = 0
    temp_area = []
    
    # Gray scale conversion
    gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply the Gaussian filter to the noise elimination
    gris = cv2.GaussianBlur(gris, (21, 21), 0)

    ## Obtain the background (initial frame)
    #if fondo is None:
    #	fondo = gris
    #	continue

    # Make the subtract between the background and the foreground
    resta = cv2.absdiff(fondo, gris)

    # Apply the threshold
    umbral = cv2.threshold(resta, 50, 500, cv2.THRESH_BINARY)[1]

    # Fill the holes
    umbral = cv2.dilate(umbral, None, iterations=3)

    # Contorn detecting
    contornosimg = umbral.copy()

    # Search contorn in the picture
    im, contornos, hierarchy = cv2.findContours(contornosimg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    # Tour all frames founds.
    for c in contornos:
        # Small contours elimination
        if cv2.contourArea(c) < 2000:
            continue

        # Get the main rectangle that englobe the object
        (x, y, w, h) = cv2.boundingRect(c)
        # Draw the rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        a = a + 1
        temp_area.append(cv2.contourArea(c))

    if a > 0:
        cont = cont + 1
        if cont > 10:
            cont = 0
            if len(lst_area) == 0:
                lst_area = temp_area
            else:
                for t in temp_area:
                    for j in lst_area:
                        meh.append(abs(t-j)/j)
                        if (abs(t-j)/j) > 1 :
                            lst_area.append(t)

    # Show the different videos
    cv2.imshow("Camara", frame)
    cv2.imshow("Umbral", umbral)
    cv2.imshow("Resta", resta)
    cv2.imshow("Contorno", contornosimg)

    # Capture a key to finish the video (s)
    key = cv2.waitKey(1) & 0xFF

    # Pause time for a good visualitation
    #time.sleep(0.015)

    # Check if the letter 's' has been pulsed
    if key == ord("s"):
        break
        
# Liberate the camera and close the videos
camara.release()
cv2.destroyAllWindows()

num_mov = len(lst_area)

if num_mov != 0:    
    print("Motion was detected")
    print("Presence of "+str(num_mov)+" subject(s) was distinguished")

Motion was detected
Presence of 1 subject(s) was distinguished


In [21]:
len(temp_area)

2

In [22]:
temp_area

[8311.0, 7641.0]

In [31]:
lst_area

[11548.0]

In [32]:
meh.sort()
meh

[0.00034638032559750607,
 0.001515413924489089,
 0.0015587114651887772,
 0.0015587114651887772,
 0.0018184967093869068,
 0.003550398337374437,
 0.004286456529269137,
 0.005325597506061656,
 0.005585382750259786,
 0.00792344994804295,
 0.008096640110841704,
 0.011170765500519571,
 0.01840145479736751,
 0.01961378593695878,
 0.02039314166955317,
 0.02164877034984413,
 0.02558884655351576,
 0.027537235885001732,
 0.029399030135088326,
 0.031217526844475235,
 0.03225666782126775,
 0.03316591617596121,
 0.03775545549012816,
 0.03775545549012816,
 0.038621406304121926,
 0.0405697956356079,
 0.043167648077589195,
 0.04554901281607205,
 0.046198475926567374,
 0.04780048493245584,
 0.04823346033945272,
 0.05290959473501905,
 0.05464149636300658,
 0.05693626602009006,
 0.0578888119154832,
 0.05897125043297541,
 0.06040006927606512,
 0.06113612746795982,
 0.06312781434014549,
 0.06524939383443021,
 0.07235019050917908,
 0.07235019050917908,
 0.07239348804987877,
 0.07256667821267752,
 0.073692414